In [1]:
################################################################################################################################
# 3 May 2021                                                                                                                   #    
# Abraham Tishelman-Charny                                                                                                     #
#                                                                                                                              #
# The purpose of this notebook is to prepare ECAL Trigger plots for CMS approval, with the aim of including in TIPP2021 poster #
################################################################################################################################

In [2]:
import uproot 
import numpy as np 
from matplotlib import pyplot as plt 

In [14]:
# f_u['samples'].keys()

[b'BX0',
 b'nPU',
 b'trueA',
 b'trueA_T',
 b'digis',
 b'E_pu',
 b'second_set_digis',
 b'second_set_signal_digis',
 b'second_set_trueA',
 b'second_set_trueA_T',
 b'first_set_t0',
 b'second_set_t0',
 b'noise_digis',
 b'signal_digis',
 b'pileup_digis',
 b'energyPU']

In [3]:
##-- Want to plot rejected spike times from a strip with OOT weights for CMS plot approval 
##-- Need input sample, 2 sets of weights to use 

f = "/eos/user/a/atishelm/ntuples/EcalL1Optimization/DoubleWeights/GradientDescent/PU50_1M_Events_3GeVminSpikes/samples/hadded.root"
f_u = uproot.open(f)

In [4]:
PUoptimized_weights_first_set = [0, 0, -0.546875, -0.546875, 0.265625, 0.484375, 0.34375, 0, 0, 0]
p5_weights_two = [0, 0, -0.546875, -0.546875, 0.171875, 0.515625, 0.421875, 0, 0, 0]
p7p5_weights_two = [0, 0, -0.546875, -0.546875, 0.109375, 0.515625, 0.453125, 0, 0, 0]
p10_weights_two = [0, 0, -0.53125, -0.53125, 0.046875, 0.53125, 0.484375, 0, 0, 0]
p12p5_weights_two = [0, 0, -0.515625, -0.515625, -0.03125, 0.53125, 0.53125, 0, 0, 0]
p15_weights_two = [0, 0, -0.484375, -0.484375, -0.125, 0.53125, 0.5625, 0, 0, 0]
p17p5_weights_two = [0, 0, -0.453125, -0.453125, -0.203125, 0.515625, 0.59375, 0, 0, 0]
p20_weights_two = [0, 0, -0.40625, -0.40625, -0.296875, 0.5, 0.609375, 0, 0, 0]
p22p5_weights_two = [0, 0, -0.359375, -0.359375, -0.359375, 0.46875, 0.609375, 0, 0, 0]

In [5]:
##-- Plot weights 

PUoptimized_weights_first_set = [0, 0, -0.546875, -0.546875, 0.265625, 0.484375, 0.34375, 0, 0, 0]
p10_weights_two = [0, 0, -0.53125, -0.53125, 0.046875, 0.53125, 0.484375, 0, 0, 0]
p15_weights_two = [0, 0, -0.484375, -0.484375, -0.125, 0.53125, 0.5625, 0, 0, 0]
p20_weights_two = [0, 0, -0.40625, -0.40625, -0.296875, 0.5, 0.609375, 0, 0, 0]

samples = [i for i in range(1,11)]

fig, ax = plt.subplots()

plt.plot(samples, PUoptimized_weights_first_set, label = "In-time", marker = '.', linestyle = '-', markersize = 12)
plt.plot(samples, p10_weights_two, label = "Out-of-time: 10ns", marker = '.', linestyle = '-', markersize = 12)
plt.plot(samples, p15_weights_two, label = "Out-of-time: 15ns", marker = '.', linestyle = '-', markersize = 12)
plt.plot(samples, p20_weights_two, label = "Out-of-time: 20ns", marker = '.', linestyle = '-', markersize = 12)

plt.title("In-time and out-of-time weight values", fontsize = 20)
plt.ylabel("Weight value", fontsize = 15)
plt.xlabel("Sample", fontsize = 15)
plt.grid()
plt.legend()
# plt.show()
plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Weight_Values_TIPP2021.png")
plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Weight_Values_TIPP2021.pdf")
plt.close()

print("DONE")

DONE


In [6]:
spike_digis = f_u['samples']['second_set_digis'].array()
spike_times = f_u['samples']['second_set_t0'].array()
spike_energies = f_u['samples']['second_set_trueA_T'].array()
signal_times = f_u['samples']['first_set_t0'].array()
rel_spike_time = np.subtract(spike_times, signal_times)
N_wfs = len(spike_digis)
print("Number of spike waveforms:",N_wfs)

Number of spike waveforms: 999600


In [9]:
time_shifts = ["p10", "p15", "p20"]

In [7]:
# time_shifts = ["p5", "p7p5", "p10", "p12p5", "p15", "p17p5", "p20", "p22p5"]
time_shifts = ["p10", "p15", "p20"]

max_wf = -1
A_1 = []
# A_2 = []

##-- Instantiate A_2's 
for time_shift in time_shifts:
    exec("A_2_%s = []"%(time_shift))

spike_times_ = []
for iwf, wf in enumerate(spike_digis):
    if(iwf%50000 == 0): print("On waveform %s"%(iwf))
    if(iwf == max_wf): break
    A_1.append(sum(wf * PUoptimized_weights_first_set))
#     A_2.append(sum(wf * p12p5_weights_two))

    ##-- Append A_2's 
    for time_shift in time_shifts:
        exec("recoA2 = sum(wf * %s_weights_two)"%(time_shift))
        exec("A_2_%s.append(recoA2)"%(time_shift))
        
    spike_times_.append(rel_spike_time[iwf])
    
A_1 = np.array(A_1)
# A_2 = np.array(A_2)

##-- List to array A_2's 
for time_shift in time_shifts:
    exec("A_2_%s = np.array(A_2_%s)"%(time_shift, time_shift))

spike_times_ = np.array(spike_times_)
spike_energies = np.array(spike_energies)

##-- A2 > A1 masks - one per time shift 
for time_shift in time_shifts:
    exec("A_2_gt_A_1_mask_%s = A_2_%s > A_1"%(time_shift, time_shift))

# A_2_gt_A_1_mask = A_2 > A_1    
    
# fig,ax = plt.subplots(1,3, figsize=(20,4))
# fig,ax = plt.subplots(figsize=(8,4))    

print("DONE")

On waveform 0
On waveform 50000
On waveform 100000
On waveform 150000
On waveform 200000
On waveform 250000
On waveform 300000
On waveform 350000
On waveform 400000
On waveform 450000
On waveform 500000
On waveform 550000
On waveform 600000
On waveform 650000
On waveform 700000
On waveform 750000
On waveform 800000
On waveform 850000
On waveform 900000
On waveform 950000
DONE


In [10]:
##-- Make a plot for each time shift

time_shift_label_dict = {
    "p5" : "+5",
    "p7p5" : "+7.5",
    "p10" : "+10",
    "p12p5" : "+12.5",
    "p15" : "+15",
    "p17p5" : "+17.5",
    "p20" : "+20",
    "p22p5" : "+22.5",
}

for time_shift in time_shifts:
    time_shift_label = time_shift_label_dict[time_shift]
    print("On time shift:",time_shift_label)

    spikeTime_min, spikeTime_max, spike_nbins = 0, 25, 400
    dt = (float(spikeTime_max) - spikeTime_min) / float(spike_nbins)
    # spikeTime_min, spikeTime_max, spike_nbins = 0, 25, 25
    spikeTimeBins = np.linspace(spikeTime_min, spikeTime_max, spike_nbins + 1)

    ##-- Overlay two plots and plot ratio 
    fig, axarr = plt.subplots(2, 
                                sharex=True, 
                                gridspec_kw={
                                    'hspace': 0.15,
    #                                     'height_ratios': (0.8,0.2)
                                    'height_ratios': (0.7,0.3)
                                    }
                                )     


    upper = axarr[0] ##-- Upper axis. Will contain two distributions, one for each CMSSW config 
    lower = axarr[1] ##-- Lower axis. Will contain ratio of two upper distributions 
    lower.grid(True)
    fig.set_size_inches(8, 6)

    ##-- Upper left plot text
    ##-- CMS 
    plt.text(
        # 0.05, 0.9, u"CMS $\it{Preliminary}$",
        0., 1., u"CMS ",
        fontsize=18, fontweight='bold',
        horizontalalignment='left',
        verticalalignment='bottom',
        transform=upper.transAxes
    )

    ##-- Preliminary 
    plt.text(
        # 0.05, 0.9, u"CMS $\it{Preliminary}$",
        0.11, 1., u"$\it{Standalone}$ $\it{Simulation}$",
        fontsize=18,
        horizontalalignment='left',
        verticalalignment='bottom',
        transform=upper.transAxes
    )

    ##-- sqrt(s)
    #     plt.text(
    #             1., 1., r"%1.0f fb$^{-1}$ (13 TeV)" % 137,
    #             fontsize=14, horizontalalignment='right',
    #             verticalalignment='bottom',
    #             transform=ax.transAxes
    #     ) 

    max_spike_time = max(spike_times_)
    
    exec("time_shift_mask = A_2_gt_A_1_mask_%s"%(time_shift))
    
    max_rej_time = max(spike_times_[time_shift_mask])
    rej_spike_times = spike_times_[time_shift_mask]

#     print("1 / max_spike_time:",1 / max_spike_time)

    # weights = [1 / max_spike_time for i in range(0,len(spike_times_))]
    # weights_rej = [1 / max_spike_time for i in range(0,len(rej_spike_times))]
    # print("weights:",weights)

    upper.hist(spike_times_,alpha=0.5,label="All",bins=spikeTimeBins, histtype = 'stepfilled')
    upper.hist(rej_spike_times,color='red',label="Tagged",bins=spikeTimeBins, histtype = 'stepfilled')
#     plt.ylabel("Entries")  
#     upper.set_ylabel("Entries / %s ns"%(str(dt)), fontsize = 15, labelpad=5)
    upper.set_ylabel("Entries", fontsize = 15, labelpad=5)
    # plt.title("Spike Peak Times", fontsize = 20)
    # plt.xlabel(r"Spike $t_{0}$ (ns)")

    upper.legend(loc='upper right', prop={'size': 14})
    upper.grid(True)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    ##-- Produce numpy histograms to obtain bin heights 
#     print("Producing numpy histograms...")
    allSpikes_bin_vals, bins_ = np.histogram(spike_times_, bins = spikeTimeBins)
    rejSpikes_bin_vals, bins_ = np.histogram(rej_spike_times, bins = spikeTimeBins)

    binvalues1_a = np.array(rejSpikes_bin_vals, dtype = float)
    binvalues2_a = np.array(allSpikes_bin_vals, dtype = float)    

    ##-- Get ratio of bins 
#     print("Taking ratio...")

    ratio = np.true_divide(binvalues1_a , binvalues2_a, out = np.zeros_like(binvalues2_a), where = binvalues2_a != 0)

    ##-- Get error on each bin ratio 
    ratio_errors = []
    for i, ratio_val in enumerate(ratio):
        val_1 = binvalues1_a[i] 
        val_2 = binvalues2_a[i]
        if(val_2 == 0): 
            ratio_errors.append(0)
            continue 
        else:
            error1 = np.sqrt(val_1)
            error2 = np.sqrt(val_2)
            ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2) 
            ratio_errors.append(ratio_error)

    ##-- Plot ratio of two histograms in lower section of canvas 
    bin_centers = [ (((float(spikeTimeBins[i+1]) - float(spikeTimeBins[i])) / 2.) + float(spikeTimeBins[i])) for i in range(0, len(spikeTimeBins)-1)]

    # lower.plot([binInfo[0],binInfo[1]],[1,1],linestyle=':', color = 'black')
    xerr = [ ((float(spikeTimeBins[i+1]) - float(spikeTimeBins[i])) / 2.) for i in range(0, len(spikeTimeBins) - 1)]  
    zero_errors = [0 for entry in ratio_errors] ##-- for no errors 
    # lower.errorbar(bin_centers, ratio, xerr = xerr , yerr = ratio_errors, marker = '.', color = 'black', ls = '') #drawstyle = 'steps-mid') ##-- For no ratio error bars 
#     lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '.', color = 'black', ls = '', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 
    lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '', color = 'black', ls = '-', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 

#     lower.fill_between(bin_centers, np.subtract(ratio, ratio_errors), np.add(ratio, ratio_errors))

    # lower.errorbar(bin_centers, ratio, xerr = xerr , yerr = ratio_errors, marker = '.', color = 'black', ls = '') #drawstyle = 'steps-mid') ##-- To add statistical ratio error bars  
    lower.set_ylim(-0.1, 1.1) ##-- Ratio ymin, ymax 
    lower.set_ylabel("Fraction tagged", fontsize = 15, labelpad=19)
    lower.set_xlabel(r"Spike time (ns)", fontsize = 15) 
    # lower.plot([spikeTime_min, spikeTime_max],[1,1],linestyle=':', color = 'black')
    
    fig.text(0.5, 0.65, " %s ns Weights"%(time_shift_label), transform=upper.transAxes, fontsize=20,
            verticalalignment='top', bbox=props)       
    # plt.savefig('%s/RejectedWaveforms_lSiglSpike_%s_%s.png'%(outDirec,lSigVal,lSpikeVal))
    # plt.savefig('%s/RejectedWaveforms_lSiglSpike_%s_%s.pdf'%(outDirec,lSigVal,lSpikeVal))
    #     plt.subplots_adjust(wspace=0.2)
    # fig.grid()
    fig.tight_layout()
    plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Double_Amplitude_Weights_SpikeTaggingwithOOTWeights_%sns.png"%(time_shift))
    plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Double_Amplitude_Weights_SpikeTaggingwithOOTWeights_%sns.pdf"%(time_shift))
#     plt.show()
    plt.close()
    
print("DONE")

On time shift: +10


<ipython-input-10-e9e6696fd81b>:117: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-10-e9e6696fd81b>:144: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


On time shift: +15


<ipython-input-10-e9e6696fd81b>:117: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-10-e9e6696fd81b>:144: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


On time shift: +20


<ipython-input-10-e9e6696fd81b>:117: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-10-e9e6696fd81b>:144: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


DONE


In [13]:
##-- Plot all time shifts on single plot 

time_shift_label_dict = {
    "p5" : "+5",
    "p7p5" : "+7.5",
    "p10" : "+10",
    "p12p5" : "+12.5",
    "p15" : "+15",
    "p17p5" : "+17.5",
    "p20" : "+20",
    "p22p5" : "+22.5",
}

fig, axarr = plt.subplots(2, 
                            sharex=True, 
                            gridspec_kw={
                                'hspace': 0.15,
#                                     'height_ratios': (0.8,0.2)
                                'height_ratios': (0.7,0.3)
                                }
                            )     


upper = axarr[0] ##-- Upper axis. Will contain two distributions, one for each CMSSW config 
lower = axarr[1] ##-- Lower axis. Will contain ratio of two upper distributions 
lower.grid(True)
fig.set_size_inches(8, 6)

##-- Upper left plot text
##-- CMS 
plt.text(
    # 0.05, 0.9, u"CMS $\it{Preliminary}$",
    0., 1., u"CMS ",
    fontsize=18, fontweight='bold',
    horizontalalignment='left',
    verticalalignment='bottom',
    transform=upper.transAxes
)

##-- Preliminary 
plt.text(
    # 0.05, 0.9, u"CMS $\it{Preliminary}$",
    0.11, 1., u"$\it{Standalone}$ $\it{Simulation}$",
    fontsize=18,
    horizontalalignment='left',
    verticalalignment='bottom',
    transform=upper.transAxes
)

spikeTime_min, spikeTime_max, spike_nbins = 0, 25, 400
dt = (float(spikeTime_max) - spikeTime_min) / float(spike_nbins)
spikeTimeBins = np.linspace(spikeTime_min, spikeTime_max, spike_nbins + 1)
max_spike_time = max(spike_times_)    
    
##-- Plot full spike time distribution 
upper.hist(spike_times_,alpha=0.5,label="All",bins=spikeTimeBins, histtype = 'stepfilled')    
    

# colors = [(165 / 255, 15 / 255, 21 / 255), (222 / 255, 45 / 255, 38 / 255), (251 / 255, 106 / 255, 74 / 255)] 
colors = [(165 / 255, 15 / 255, 21 / 255), (222 / 255, 45 / 255, 38 / 255), (251 / 255, 106 / 255, 74 / 255)] 
# colors = ['0', '1', '2', '3']
    
##-- Add each time shift rejection to plot 
for time_i, time_shift in enumerate(time_shifts):
    color = colors[time_i]
        
    time_shift_label = time_shift_label_dict[time_shift]
    print("On time shift:",time_shift_label)
    
    ##-- Get time shift mask for a given time shift 
    exec("time_shift_mask = A_2_gt_A_1_mask_%s"%(time_shift))
    
    max_rej_time = max(spike_times_[time_shift_mask])
    rej_spike_times = spike_times_[time_shift_mask]

#     print("1 / max_spike_time:",1 / max_spike_time)

    # weights = [1 / max_spike_time for i in range(0,len(spike_times_))]
    # weights_rej = [1 / max_spike_time for i in range(0,len(rej_spike_times))]
    # print("weights:",weights)

#     upper.hist(spike_times_,alpha=0.5,label="All",bins=spikeTimeBins, histtype = 'stepfilled')
    upper.hist(rej_spike_times,color=color,label="%s ns weights Tagged"%(time_shift_label),bins=spikeTimeBins, histtype = 'stepfilled')
#     upper.hist(rej_spike_times,label="%s ns weights Tagged"%(time_shift_label),bins=spikeTimeBins, histtype = 'stepfilled')
#     plt.ylabel("Entries")  
#     upper.set_ylabel("Entries / %s ns"%(str(dt)), fontsize = 15, labelpad=5)
    upper.set_ylabel("Entries", fontsize = 15, labelpad=5)
    # plt.title("Spike Peak Times", fontsize = 20)
    # plt.xlabel(r"Spike $t_{0}$ (ns)")

    upper.legend(loc='upper right', prop={'size': 14})
    upper.grid(True)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    ##-- Produce numpy histograms to obtain bin heights 
#     print("Producing numpy histograms...")
    allSpikes_bin_vals, bins_ = np.histogram(spike_times_, bins = spikeTimeBins)
    rejSpikes_bin_vals, bins_ = np.histogram(rej_spike_times, bins = spikeTimeBins)

    binvalues1_a = np.array(rejSpikes_bin_vals, dtype = float)
    binvalues2_a = np.array(allSpikes_bin_vals, dtype = float)    

    ##-- Get ratio of bins 
#     print("Taking ratio...")

    ratio = np.true_divide(binvalues1_a , binvalues2_a, out = np.zeros_like(binvalues2_a), where = binvalues2_a != 0)

    ##-- Get error on each bin ratio 
    ratio_errors = []
    for i, ratio_val in enumerate(ratio):
        val_1 = binvalues1_a[i] 
        val_2 = binvalues2_a[i]
        if(val_2 == 0): 
            ratio_errors.append(0)
            continue 
        else:
            error1 = np.sqrt(val_1)
            error2 = np.sqrt(val_2)
            ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2) 
            ratio_errors.append(ratio_error)

    ##-- Plot ratio of two histograms in lower section of canvas 
    bin_centers = [ (((float(spikeTimeBins[i+1]) - float(spikeTimeBins[i])) / 2.) + float(spikeTimeBins[i])) for i in range(0, len(spikeTimeBins)-1)]

    # lower.plot([binInfo[0],binInfo[1]],[1,1],linestyle=':', color = 'black')
    xerr = [ ((float(spikeTimeBins[i+1]) - float(spikeTimeBins[i])) / 2.) for i in range(0, len(spikeTimeBins) - 1)]  
    zero_errors = [0 for entry in ratio_errors] ##-- for no errors 
    # lower.errorbar(bin_centers, ratio, xerr = xerr , yerr = ratio_errors, marker = '.', color = 'black', ls = '') #drawstyle = 'steps-mid') ##-- For no ratio error bars 
#     lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '.', color = 'black', ls = '', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 
    lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '', color = color, ls = '-', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 

#     lower.fill_between(bin_centers, np.subtract(ratio, ratio_errors), np.add(ratio, ratio_errors))

    # lower.errorbar(bin_centers, ratio, xerr = xerr , yerr = ratio_errors, marker = '.', color = 'black', ls = '') #drawstyle = 'steps-mid') ##-- To add statistical ratio error bars  
    lower.set_ylim(-0.1, 1.1) ##-- Ratio ymin, ymax 
    lower.set_ylabel("Fraction tagged", fontsize = 15, labelpad=19)
    lower.set_xlabel(r"Spike time (ns)", fontsize = 15) 
    # lower.plot([spikeTime_min, spikeTime_max],[1,1],linestyle=':', color = 'black')
    
#     fig.text(0.5, 0.65, " %s ns Weights"%(time_shift_label), transform=upper.transAxes, fontsize=20,
#             verticalalignment='top', bbox=props)       
    # plt.savefig('%s/RejectedWaveforms_lSiglSpike_%s_%s.png'%(outDirec,lSigVal,lSpikeVal))
    # plt.savefig('%s/RejectedWaveforms_lSiglSpike_%s_%s.pdf'%(outDirec,lSigVal,lSpikeVal))
    #     plt.subplots_adjust(wspace=0.2)
    # fig.grid()
    
fig.tight_layout()
plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Double_Amplitude_Weights_SpikeTaggingwithOOTWeights_allShifts.png")
plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Double_Amplitude_Weights_SpikeTaggingwithOOTWeights_allShifts.pdf")
plt.close()
    
print("DONE")

On time shift: +10


<ipython-input-13-ae0fe146ec24>:119: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-13-ae0fe146ec24>:119: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)


On time shift: +15
On time shift: +20


<ipython-input-13-ae0fe146ec24>:119: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-13-ae0fe146ec24>:147: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


DONE


In [12]:
##-- Plot all time shifts on single plot 

time_shift_label_dict = {
    "p5" : "+5",
    "p7p5" : "+7.5",
    "p10" : "+10",
    "p12p5" : "+12.5",
    "p15" : "+15",
    "p17p5" : "+17.5",
    "p20" : "+20",
    "p22p5" : "+22.5",
}

fig, axarr = plt.subplots(2, 
                            sharex=True, 
                            gridspec_kw={
                                'hspace': 0.15,
#                                     'height_ratios': (0.8,0.2)
                                'height_ratios': (0.7,0.3)
                                }
                            )     


upper = axarr[0] ##-- Upper axis. Will contain two distributions, one for each CMSSW config 
lower = axarr[1] ##-- Lower axis. Will contain ratio of two upper distributions 
lower.grid(True)
fig.set_size_inches(8, 6)

##-- Upper left plot text
##-- CMS 
plt.text(
    # 0.05, 0.9, u"CMS $\it{Preliminary}$",
    0., 1., u"CMS ",
    fontsize=18, fontweight='bold',
    horizontalalignment='left',
    verticalalignment='bottom',
    transform=upper.transAxes
)

##-- Preliminary 
plt.text(
    # 0.05, 0.9, u"CMS $\it{Preliminary}$",
    0.11, 1., u"$\it{Standalone}$ $\it{Simulation}$",
    fontsize=18,
    horizontalalignment='left',
    verticalalignment='bottom',
    transform=upper.transAxes
)

spikeTime_min, spikeTime_max, spike_nbins = 0, 25, 400
dt = (float(spikeTime_max) - spikeTime_min) / float(spike_nbins)
spikeTimeBins = np.linspace(spikeTime_min, spikeTime_max, spike_nbins + 1)
max_spike_time = max(spike_times_)    
    
##-- Plot full spike time distribution 
upper.hist(spike_times_,alpha=0.5,label="All",bins=spikeTimeBins, histtype = 'stepfilled')    
    

# colors = [(165 / 255, 15 / 255, 21 / 255), (222 / 255, 45 / 255, 38 / 255), (251 / 255, 106 / 255, 74 / 255)] 
# colors = [(165 / 255, 15 / 255, 21 / 255), (222 / 255, 45 / 255, 38 / 255), (251 / 255, 106 / 255, 74 / 255)] 
# colors = ['0', '1', '2', '3']
colors = ["C%s"%(str(i)) for i in range(0,10)]
    
##-- Add each time shift rejection to plot 
for time_i, time_shift in enumerate(time_shifts):
    color = colors[time_i]
        
    time_shift_label = time_shift_label_dict[time_shift]
    print("On time shift:",time_shift_label)
    
    ##-- Get time shift mask for a given time shift 
    exec("time_shift_mask = A_2_gt_A_1_mask_%s"%(time_shift))
    
    max_rej_time = max(spike_times_[time_shift_mask])
    rej_spike_times = spike_times_[time_shift_mask]

#     print("1 / max_spike_time:",1 / max_spike_time)

    # weights = [1 / max_spike_time for i in range(0,len(spike_times_))]
    # weights_rej = [1 / max_spike_time for i in range(0,len(rej_spike_times))]
    # print("weights:",weights)

#     upper.hist(spike_times_,alpha=0.5,label="All",bins=spikeTimeBins, histtype = 'stepfilled')
    upper.hist(rej_spike_times,color=color,label="%s ns weights Tagged"%(time_shift_label),bins=spikeTimeBins, histtype = 'stepfilled')
#     upper.hist(rej_spike_times,label="%s ns weights Tagged"%(time_shift_label),bins=spikeTimeBins, histtype = 'stepfilled')
#     plt.ylabel("Entries")  
    upper.set_ylabel("Entries / %s ns"%(str(dt)), fontsize = 15, labelpad=5)
    # plt.title("Spike Peak Times", fontsize = 20)
    # plt.xlabel(r"Spike $t_{0}$ (ns)")

#     upper.legend(loc='upper right', prop={'size': 14})
    upper.legend(loc='upper right', prop={'size': 10})
    upper.grid(True)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    ##-- Produce numpy histograms to obtain bin heights 
#     print("Producing numpy histograms...")
    allSpikes_bin_vals, bins_ = np.histogram(spike_times_, bins = spikeTimeBins)
    rejSpikes_bin_vals, bins_ = np.histogram(rej_spike_times, bins = spikeTimeBins)

    binvalues1_a = np.array(rejSpikes_bin_vals, dtype = float)
    binvalues2_a = np.array(allSpikes_bin_vals, dtype = float)    

    ##-- Get ratio of bins 
#     print("Taking ratio...")

    ratio = np.true_divide(binvalues1_a , binvalues2_a, out = np.zeros_like(binvalues2_a), where = binvalues2_a != 0)

    ##-- Get error on each bin ratio 
    ratio_errors = []
    for i, ratio_val in enumerate(ratio):
        val_1 = binvalues1_a[i] 
        val_2 = binvalues2_a[i]
        if(val_2 == 0): 
            ratio_errors.append(0)
            continue 
        else:
            error1 = np.sqrt(val_1)
            error2 = np.sqrt(val_2)
            ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2) 
            ratio_errors.append(ratio_error)

    ##-- Plot ratio of two histograms in lower section of canvas 
    bin_centers = [ (((float(spikeTimeBins[i+1]) - float(spikeTimeBins[i])) / 2.) + float(spikeTimeBins[i])) for i in range(0, len(spikeTimeBins)-1)]

    # lower.plot([binInfo[0],binInfo[1]],[1,1],linestyle=':', color = 'black')
    xerr = [ ((float(spikeTimeBins[i+1]) - float(spikeTimeBins[i])) / 2.) for i in range(0, len(spikeTimeBins) - 1)]  
    zero_errors = [0 for entry in ratio_errors] ##-- for no errors 
    # lower.errorbar(bin_centers, ratio, xerr = xerr , yerr = ratio_errors, marker = '.', color = 'black', ls = '') #drawstyle = 'steps-mid') ##-- For no ratio error bars 
#     lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '.', color = 'black', ls = '', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 
#     lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '', color = color, ls = '-', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 
    lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '', ls = '-', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 

#     lower.fill_between(bin_centers, np.subtract(ratio, ratio_errors), np.add(ratio, ratio_errors))

    # lower.errorbar(bin_centers, ratio, xerr = xerr , yerr = ratio_errors, marker = '.', color = 'black', ls = '') #drawstyle = 'steps-mid') ##-- To add statistical ratio error bars  
    lower.set_ylim(-0.1, 1.1) ##-- Ratio ymin, ymax 
    lower.set_ylabel("Fraction tagged", fontsize = 15, labelpad=19)
    lower.set_xlabel(r"Spike time (ns)", fontsize = 15) 
    # lower.plot([spikeTime_min, spikeTime_max],[1,1],linestyle=':', color = 'black')
    
#     fig.text(0.5, 0.65, " %s ns Weights"%(time_shift_label), transform=upper.transAxes, fontsize=20,
#             verticalalignment='top', bbox=props)       
    # plt.savefig('%s/RejectedWaveforms_lSiglSpike_%s_%s.png'%(outDirec,lSigVal,lSpikeVal))
    # plt.savefig('%s/RejectedWaveforms_lSiglSpike_%s_%s.pdf'%(outDirec,lSigVal,lSpikeVal))
    #     plt.subplots_adjust(wspace=0.2)
    # fig.grid()
    
fig.tight_layout()
plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Double_Amplitude_Weights_SpikeTaggingwithOOTWeights_allShifts.png")
plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Double_Amplitude_Weights_SpikeTaggingwithOOTWeights_allShifts.pdf")
plt.close()
    
print("DONE")

On time shift: +5
On time shift: +7.5


<ipython-input-12-fba95ecba94f>:120: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)


On time shift: +10


<ipython-input-12-fba95ecba94f>:120: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)


On time shift: +12.5


...

On time shift: +15
On time shift: +17.5
On time shift: +20


<ipython-input-12-fba95ecba94f>:120: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-12-fba95ecba94f>:120: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)


On time shift: +22.5


<ipython-input-12-fba95ecba94f>:120: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-12-fba95ecba94f>:149: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


DONE


In [28]:
##-- Same plot but for spike energy instead of time 

time_shift_label_dict = {
    "p5" : "+5",
    "p7p5" : "+7.5",
    "p10" : "+10",
    "p12p5" : "+12.5",
    "p15" : "+15",
    "p17p5" : "+17.5",
    "p20" : "+20",
    "p22p5" : "+22.5",
}

fig, axarr = plt.subplots(2, 
                            sharex=True, 
                            gridspec_kw={
                                'hspace': 0.15,
#                                     'height_ratios': (0.8,0.2)
                                'height_ratios': (0.7,0.3)
                                }
                            )     


upper = axarr[0] ##-- Upper axis. Will contain two distributions, one for each CMSSW config 
lower = axarr[1] ##-- Lower axis. Will contain ratio of two upper distributions 
lower.grid(True)
upper.set_yscale('log')
upper.set_ylim(1, 10e7)
fig.set_size_inches(8, 6)

##-- Upper left plot text
##-- CMS 
plt.text(
    # 0.05, 0.9, u"CMS $\it{Preliminary}$",
    0., 1., u"CMS ",
    fontsize=18, fontweight='bold',
    horizontalalignment='left',
    verticalalignment='bottom',
    transform=upper.transAxes
)

##-- Preliminary 
plt.text(
    # 0.05, 0.9, u"CMS $\it{Preliminary}$",
    0.11, 1., u"$\it{Standalone}$ $\it{Simulation}$",
    fontsize=18,
    horizontalalignment='left',
    verticalalignment='bottom',
    transform=upper.transAxes
)

spikeTime_min, spikeTime_max, spike_nbins = 0, 25, 400
spikeEnergy_min, spikeEnergy_max, spikeEnergy_nbins = 3, 100, 97

dt = (float(spikeTime_max) - spikeTime_min) / float(spike_nbins)
spikeTimeBins = np.linspace(spikeTime_min, spikeTime_max, spike_nbins + 1)

spikeEnergyBins = np.linspace(spikeEnergy_min, spikeEnergy_max, spikeEnergy_nbins + 1)
dE = (float(spikeEnergy_max) - spikeEnergy_min) / float(spikeEnergy_nbins)

# max_spike_time = max(spike_times_)    
max_spike_energy = max(spike_energies)
    
##-- Plot full spike time distribution 
# upper.hist(spike_times_,alpha=0.5,label="All",bins=spikeTimeBins, histtype = 'stepfilled')    
upper.hist(spike_energies,alpha=0.5,label="All",bins=spikeEnergyBins, histtype = 'stepfilled')    
    

# colors = [(165 / 255, 15 / 255, 21 / 255), (222 / 255, 45 / 255, 38 / 255), (251 / 255, 106 / 255, 74 / 255)] 
# colors = [(165 / 255, 15 / 255, 21 / 255), (222 / 255, 45 / 255, 38 / 255), (251 / 255, 106 / 255, 74 / 255)] 
# colors = ['0', '1', '2', '3']
colors = ["C%s"%(str(i)) for i in range(0,10)]
    
##-- Add each time shift rejection to plot 
for time_i, time_shift in enumerate(time_shifts):
    color = colors[time_i]
        
    time_shift_label = time_shift_label_dict[time_shift]
    print("On time shift:",time_shift_label)
    
    ##-- Get time shift mask for a given time shift 
    exec("time_shift_mask = A_2_gt_A_1_mask_%s"%(time_shift))
    
    max_rej_time = max(spike_energies[time_shift_mask])
    rej_spike_times = spike_energies[time_shift_mask]
    
    max_rej_Energy = max(spike_energies[time_shift_mask])
    rej_spike_Energies = spike_energies[time_shift_mask]    

#     print("1 / max_spike_time:",1 / max_spike_time)

    # weights = [1 / max_spike_time for i in range(0,len(spike_times_))]
    # weights_rej = [1 / max_spike_time for i in range(0,len(rej_spike_times))]
    # print("weights:",weights)

#     upper.hist(spike_times_,alpha=0.5,label="All",bins=spikeTimeBins, histtype = 'stepfilled')
    upper.hist(rej_spike_Energies,color=color,label="%s ns weights Tagged"%(time_shift_label),bins=spikeEnergyBins, histtype = 'stepfilled')
#     upper.hist(rej_spike_times,label="%s ns weights Tagged"%(time_shift_label),bins=spikeTimeBins, histtype = 'stepfilled')
#     plt.ylabel("Entries")  
    upper.set_ylabel("Entries / %s"%(str(dE)), fontsize = 15, labelpad=5)
    # plt.title("Spike Peak Times", fontsize = 20)
    # plt.xlabel(r"Spike $t_{0}$ (ns)")

#     upper.legend(loc='upper right', prop={'size': 14})
    upper.legend(loc='upper right', prop={'size': 8})
    upper.grid(True)
#     upper.yscale('log')
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    ##-- Produce numpy histograms to obtain bin heights 
#     print("Producing numpy histograms...")
    allSpikes_bin_vals, bins_ = np.histogram(spike_energies, bins = spikeEnergyBins)
    rejSpikes_bin_vals, bins_ = np.histogram(rej_spike_Energies, bins = spikeEnergyBins)

    binvalues1_a = np.array(rejSpikes_bin_vals, dtype = float)
    binvalues2_a = np.array(allSpikes_bin_vals, dtype = float)    

    ##-- Get ratio of bins 
#     print("Taking ratio...")

    ratio = np.true_divide(binvalues1_a , binvalues2_a, out = np.zeros_like(binvalues2_a), where = binvalues2_a != 0)

    ##-- Get error on each bin ratio 
    ratio_errors = []
    for i, ratio_val in enumerate(ratio):
        val_1 = binvalues1_a[i] 
        val_2 = binvalues2_a[i]
        if(val_2 == 0): 
            ratio_errors.append(0)
            continue 
        else:
            error1 = np.sqrt(val_1)
            error2 = np.sqrt(val_2)
            ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2) 
            ratio_errors.append(ratio_error)

    ##-- Plot ratio of two histograms in lower section of canvas 
    bin_centers = [ (((float(spikeEnergyBins[i+1]) - float(spikeEnergyBins[i])) / 2.) + float(spikeEnergyBins[i])) for i in range(0, len(spikeEnergyBins)-1)]

    # lower.plot([binInfo[0],binInfo[1]],[1,1],linestyle=':', color = 'black')
    xerr = [ ((float(spikeEnergyBins[i+1]) - float(spikeEnergyBins[i])) / 2.) for i in range(0, len(spikeEnergyBins) - 1)]  
    zero_errors = [0 for entry in ratio_errors] ##-- for no errors 
    # lower.errorbar(bin_centers, ratio, xerr = xerr , yerr = ratio_errors, marker = '.', color = 'black', ls = '') #drawstyle = 'steps-mid') ##-- For no ratio error bars 
#     lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '.', color = 'black', ls = '', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 
#     lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '', color = color, ls = '-', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 
    lower.errorbar(bin_centers, ratio, xerr = zero_errors , yerr = zero_errors, marker = '', ls = '-', markersize = 1) #drawstyle = 'steps-mid') ##-- For no ratio error bars 

#     lower.fill_between(bin_centers, np.subtract(ratio, ratio_errors), np.add(ratio, ratio_errors))

    # lower.errorbar(bin_centers, ratio, xerr = xerr , yerr = ratio_errors, marker = '.', color = 'black', ls = '') #drawstyle = 'steps-mid') ##-- To add statistical ratio error bars  
    lower.set_ylim(-0.1, 1.1) ##-- Ratio ymin, ymax 
    lower.set_ylabel("Fraction tagged", fontsize = 15, labelpad=19)
    lower.set_xlabel(r"Spike energy", fontsize = 15) 
    # lower.plot([spikeTime_min, spikeTime_max],[1,1],linestyle=':', color = 'black')
      
fig.tight_layout()
plt.xscale('log')
# upper.yscale('log')
plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Double_Amplitude_Weights_SpikeTaggingwithOOTWeights_rejectedEnergies_allShifts.png")
plt.savefig("/eos/user/a/atishelm/www/EcalL1Optimization/May_2021_TIPP_Approval/Double_Amplitude_Weights_SpikeTaggingwithOOTWeights_rejectedEnergies_allShifts.pdf")
plt.close()
    
print("DONE")

On time shift: +5
On time shift: +7.5


<ipython-input-28-022fcd1288ba>:134: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)


On time shift: +10


<ipython-input-28-022fcd1288ba>:134: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)


On time shift: +12.5


...

On time shift: +15


<ipython-input-28-022fcd1288ba>:134: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-28-022fcd1288ba>:134: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)


On time shift: +17.5
On time shift: +20


<ipython-input-28-022fcd1288ba>:134: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-28-022fcd1288ba>:134: RuntimeWarning: invalid value encountered in double_scalars
  ratio_error = ratio_val * np.sqrt((error1/val_1)**2 + (error2/val_2)**2)
<ipython-input-28-022fcd1288ba>:156: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


On time shift: +22.5
DONE
